# Bacteria model to apply CAM

## Librairies

In [1]:
# --- Utils librairies ---
import os, sys
import os.path
import numpy as np
import torch
from datasets import spectral_dataloader
from training import run_epoch
from torch import optim
from time import time
from resnetCAM import ResNetCAM
from training import get_predictions

## Train model

### Load data

In [2]:
X_fn = './data/X_reference.npy'
y_fn = './data/y_reference.npy'
X = np.load(X_fn)
y = np.load(y_fn)

### Train and save model

In [3]:
# CNN parameters
layers = 6
hidden_size = 100
block_size = 2
hidden_sizes = [hidden_size] * layers
num_blocks = [block_size] * layers
input_dim = 1000
in_channels = 64
n_classes = 30
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="1,2,3,4";
cuda = torch.cuda.is_available()

In [4]:
cnn = ResNetCAM(hidden_sizes, num_blocks, input_dim=input_dim,
                in_channels=in_channels, n_classes=n_classes)
if cuda: cnn.cuda()

In [5]:
p_val = 0.1
n_val = int(60000 * p_val)
idx_tr = list(range(60000))
np.random.shuffle(idx_tr)
idx_val = idx_tr[:n_val]
idx_tr = idx_tr[n_val:]

In [6]:
epochs = 200
batch_size = 10
t0 = time()
# Set up Adam optimizer
optimizer = optim.Adam(cnn.parameters(), lr=1e-3, betas=(0.5, 0.999))
# Set up dataloaders
dl_tr = spectral_dataloader(X, y, idxs=idx_tr,
    batch_size=batch_size, shuffle=True)
dl_val = spectral_dataloader(X, y, idxs=idx_val,
    batch_size=batch_size, shuffle=False)

best_val = 0
no_improvement = 0
max_no_improvement = 20
print('Starting training!')
for epoch in range(epochs):
    print(' Epoch {}: {:0.2f}s'.format(epoch+1, time()-t0))
    # Train
    acc_tr, loss_tr = run_epoch(epoch, cnn, dl_tr, cuda,
        training=True, optimizer=optimizer)
    print('  Train acc: {:0.2f}'.format(acc_tr))
    # Val
    acc_val, loss_val = run_epoch(epoch, cnn, dl_val, cuda,
        training=False, optimizer=optimizer)
    print('  Val acc  : {:0.2f}'.format(acc_val))
    # Check performance for early stopping
    if acc_val > best_val or epoch == 0:
        best_val = acc_val
        no_improvement = 0
        torch.save(cnn.state_dict(), "CAMModel.ckpt")
    else:
        no_improvement += 1
    if no_improvement >= max_no_improvement:
        print('Finished after {} epochs!'.format(epoch+1))
        break

print('\n This training was completed in: {:0.2f}s'.format(time()-t0))

Starting training!
 Epoch 1: 0.00s
  Train acc: 83.13
  Val acc  : 87.80
 Epoch 2: 180.48s
  Train acc: 90.64
  Val acc  : 91.57
 Epoch 3: 352.38s
  Train acc: 92.10
  Val acc  : 92.60
 Epoch 4: 530.45s


KeyboardInterrupt: 

## Fine tune

### Load data

In [ ]:
X_fn = './data/X_finetune.npy'
y_fn = './data/y_finetune.npy'
X = np.load(X_fn)
y = np.load(y_fn)

### Fine-tune the pretrained model

In [ ]:
# Load trained weights
cnn = ResNetCAM(hidden_sizes, num_blocks, input_dim=input_dim,
                in_channels=in_channels, n_classes=n_classes)
if cuda: cnn.cuda()
cnn.load_state_dict(torch.load(
    'CAMModel.ckpt'))

<All keys matched successfully>

In [ ]:
p_val = 0.1
n_val = int(3000 * p_val)
idx_tr = list(range(3000))
np.random.shuffle(idx_tr)
idx_val = idx_tr[:n_val]
idx_tr = idx_tr[n_val:]

In [ ]:
# Fine-tune CNN
epochs = 30
batch_size = 10
t0 = time()
# Set up Adam optimizer
optimizer = optim.Adam(cnn.parameters(), lr=1e-3, betas=(0.5, 0.999))
# Set up dataloaders
dl_tr = spectral_dataloader(X, y, idxs=idx_tr,
    batch_size=batch_size, shuffle=True)
dl_val = spectral_dataloader(X, y, idxs=idx_val,
    batch_size=batch_size, shuffle=False)
# Fine-tune CNN for first fold
best_val = 0
no_improvement = 0
max_no_improvement = 5
print('Starting fine-tuning!')
for epoch in range(epochs):
    print(' Epoch {}: {:0.2f}s'.format(epoch+1, time()-t0))
    # Train
    acc_tr, loss_tr = run_epoch(epoch, cnn, dl_tr, cuda,
        training=True, optimizer=optimizer)
    print('  Train acc: {:0.2f}'.format(acc_tr))
    # Val
    acc_val, loss_val = run_epoch(epoch, cnn, dl_val, cuda,
        training=False, optimizer=optimizer)
    print('  Val acc  : {:0.2f}'.format(acc_val))
    # Check performance for early stopping
    if acc_val > best_val or epoch == 0:
        best_val = acc_val
        no_improvement = 0
        torch.save(cnn.state_dict(), "CAMModelFine-Tuned.ckpt")
    else:
        no_improvement += 1
    if no_improvement >= max_no_improvement:
        print('Finished after {} epochs!'.format(epoch+1))
        break

print('\n This fine-tuning was completed in: {:0.2f}s'.format(time()-t0))

Starting fine-tuning!
 Epoch 1: 0.01s
  Train acc: 3.56
  Val acc  : 1.00
 Epoch 2: 8.94s
  Train acc: 3.59
  Val acc  : 1.00
 Epoch 3: 17.86s
  Train acc: 3.59
  Val acc  : 1.00
 Epoch 4: 26.66s
  Train acc: 3.59
  Val acc  : 1.00
 Epoch 5: 35.49s
  Train acc: 3.59
  Val acc  : 1.00
 Epoch 6: 44.43s
  Train acc: 3.59
  Val acc  : 1.00
Finished after 6 epochs!

 This fine-tuning was completed in: 53.24s


## Test model

### Load data

In [ ]:
X_fn = './data_test/X_test.npy'
y_fn = './data_test/y_test.npy'
X = np.load(X_fn)
y = np.load(y_fn)

### Test no-finetuned model

In [ ]:
# Load trained weights
cnn = ResNetCAM(hidden_sizes, num_blocks, input_dim=input_dim,
                in_channels=in_channels, n_classes=n_classes)
if cuda: cnn.cuda()
cnn.load_state_dict(torch.load(
    'CAMModel.ckpt'))

<All keys matched successfully>

In [ ]:
# Make predictions on subset of data
t0 = time()
dl = spectral_dataloader(X, y, batch_size=10, shuffle=False)
y_hat = get_predictions(cnn, dl, cuda)
print('Predicted {} spectra: {:0.2f}s'.format(len(y_hat), time()-t0))

Predicted 3000 spectra: 3.78s


In [ ]:
# Computing accuracy
acc = (y_hat == y).mean()
print('Accuracy: {:0.1f}%'.format(100*acc))

Accuracy: 3.3%


### Test finetuned model

In [ ]:
# Load trained weights
cnn = ResNetCAM(hidden_sizes, num_blocks, input_dim=input_dim,
                in_channels=in_channels, n_classes=n_classes)
if cuda: cnn.cuda()
cnn.load_state_dict(torch.load(
    'CAMModelFine-Tuned.ckpt'))

<All keys matched successfully>

In [ ]:
# Make predictions on subset of data
t0 = time()
dl = spectral_dataloader(X, y, batch_size=10, shuffle=False)
y_hat = get_predictions(cnn, dl, cuda)
print('Predicted {} spectra: {:0.2f}s'.format(len(y_hat), time()-t0))

Before softmax : tensor([[-4.1684e+01, -7.7846e+01, -1.1979e+02, -1.0058e+02, -4.9106e+01,
         -2.1535e+02, -1.0690e+02, -1.3922e+02, -5.3918e+01, -2.5329e+01,
         -4.9596e+01,  5.4858e+02, -6.8486e+01,  7.1080e+00, -4.6915e+01,
         -1.8406e+02, -1.3403e+02, -5.8070e+01,  6.9636e+01, -4.7271e+01,
         -7.7429e+01, -1.4074e+02, -2.2342e+01, -1.2652e+02, -6.6641e+01,
         -4.6733e+01, -1.6278e+02, -2.5820e+02, -1.0772e+02, -7.7266e+01],
        [-1.1992e+02, -9.8819e+01, -1.2652e+02, -1.1056e+02, -5.7808e+01,
         -1.8561e+02, -8.8394e+01, -1.3449e+02, -5.8688e+01, -2.8548e+01,
         -1.9344e+01,  5.6398e+02, -7.2709e+01,  6.1813e+01, -2.9805e+01,
         -1.8512e+02, -1.4136e+02, -4.5519e+01,  1.0080e+02, -4.2236e+01,
         -8.3040e+01, -1.4543e+02, -1.4596e+01, -1.1811e+02, -7.3043e+01,
         -4.4813e+01, -1.6370e+02, -3.4875e+02, -1.0672e+02, -8.4708e+01],
        [-1.1571e+02, -8.9590e+01, -1.1511e+02, -7.5107e+01, -3.6053e+01,
         -1.9698e+0

/home/newuser/bacteria/resnetCAM.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  z = F.softmax(z)


tensor([[-201.5069, -103.4144, -116.6959,  -90.8227,  -47.0160, -215.1331,
         -126.5241, -140.8148,  -56.0044,  -22.4567,   49.5999,  601.2568,
          -75.6145,  -17.7970,  -44.0968, -205.5510, -151.0025,  -54.3442,
           55.4560,  -45.4249,  -84.4943, -138.1904,  -18.1461, -133.1242,
          -81.9985,  -52.9021, -164.4384, -304.3280, -109.7014,  -85.0611],
        [-301.2397, -152.3869, -162.6364, -124.1213,  -84.3237, -300.7858,
         -121.9184, -176.5936,  -75.6773,  -42.6058,   70.5084,  784.5712,
         -108.6022,  -17.2058,  -27.1288, -275.0583, -182.5881,  -55.8370,
          133.9131,  -48.7207, -114.1379, -170.6892,  -25.8319, -147.8874,
         -109.0335,  -60.7481, -199.7365, -475.9058, -136.4384, -126.6453],
        [ -40.2682,  -57.1155,  -96.3481,  -86.0029,  -17.1345, -210.1863,
          -89.5586, -121.3552,  -52.2746,  -15.5794,  -11.0563,  512.4568,
          -61.9955,  -45.0040,  -37.9256, -161.9360, -119.6817,  -48.8761,
            8.7788,  -3

In [ ]:
# Computing accuracy
acc = (y_hat == y).mean()
print('Accuracy: {:0.1f}%'.format(100*acc))

Accuracy: 3.3%
